# Script to get true ranks for model evaluation

- Modify the user query
- Click the play button, wait for completion, and scroll down to interface
- Click "here" text to be directed to the true rank response form page
- Complete and submit form


In [4]:
# ----------------------------------------------------- #

user_query = "Are there any dandisets that include EEG data or maybe something that is similar?" # Edit query string

# ----------------------------------------------------- #

import os
JOTFORM_API_KEY: str = os.getenv('JOTFORM_API_KEY', None)

import sys
sys.path.append("..")

import json
import urllib.parse
from rest.clients.qdrant import QdrantClient
from rest.clients.openai import OpenaiClient
from dotenv import load_dotenv
from IPython.display import display, clear_output
from jotform import JotformAPIClient
import ipywidgets as widgets


# Number of similar results to fetch (adjust based on model)
TOP_K = 6

# Semantic "simple" search vs keyword-based "keyword" search (adjust based on model)
METHOD = "simple"

# Populate Qdrant
def populate_qdrant():
    with open("../data/qdrant_points.json", "r") as file:
        emb = json.load(file)
    load_dotenv()
    qdrant_client = QdrantClient(host="https://906c3b3f-d3ff-4497-905f-2d7089487cf9.us-east4-0.gcp.cloud.qdrant.io")
    qdrant_client.create_collection(collection_name="dandi_collection")
    qdrant_client.add_points_to_collection(collection_name="dandi_collection", embeddings_objects=emb)
    return qdrant_client


# Get semantic search results
def get_similar_results(query: str, qdrant_client: QdrantClient):
    similar_results = qdrant_client.query_from_user_input(text=query, collection_name="dandi_collection", top_k=TOP_K)
    return similar_results

def get_keyword_results(query: str, openai_client: OpenaiClient, qdrant_client: QdrantClient):
    keywords = openai_client.keywords_extraction(user_input=query)
    similar_results = get_similar_results(query="".join(keywords), qdrant_client=qdrant_client)
    return similar_results

qdrant_client = populate_qdrant()
if METHOD == "simple":
    similar_results = get_similar_results(user_query, qdrant_client)
else:
    openai_client = OpenaiClient()
    similar_results = get_keyword_results(user_query, openai_client, qdrant_client)
similar_results = [id for id, _ in similar_results]
print(f"Similar dandiset results via {METHOD}-based search collected")
clear_output()


# Create formatted HTML blocks for Jotform ordered list
formatted_link_blocks = []
for dandiset_result in similar_results:
    dandiset_path = dandiset_result.split(":")[-1]
    dandiset_archive_link = f"https://dandiarchive.org/dandiset/{dandiset_path}"
    html_block = f"<a href='{dandiset_archive_link}' target='_blank'>{dandiset_result}</a>"
    formatted_link_blocks.append(html_block)


# Jotform client
jotform_client = JotformAPIClient(JOTFORM_API_KEY)

FORM_ID = "232963492422055" # true rank response form id
response_form = jotform_client.get_form(FORM_ID) 

USER_QUERY_NAME = "chosenQuery"
ORDERED_LIST_NAME = "typeA10"
formatted_orderable_list_values = ",".join(formatted_link_blocks)

# Parse user query for jotform URL
url_encoded_user_query = urllib.parse.quote(user_query)

# Generate jotform link for unique query results
prefill_jotform__url = f"https://form.jotform.com/{FORM_ID}?{USER_QUERY_NAME}={url_encoded_user_query.strip()}&{ORDERED_LIST_NAME}={formatted_orderable_list_values}"
link_display_html = widgets.HTML(f"""Click <a href="{prefill_jotform__url}">here</a> to open the true rank response form.""")
display(link_display_html)

HTML(value='Click <a href="https://form.jotform.com/232963492422055?chosenQuery=Are%20there%20any%20dandisets%…